In [ ]:
import os as _os
_os.chdir('..')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from operator import eq, sub
import scipy.stats
import scipy as sp
import seaborn as sns
from lib.pandas_util import idxwhere

In [ ]:
species_taxonomy = pd.read_table('ref/gtpro/species_taxonomy_ext.tsv', names=['genome_id', 'species_id', 'taxonomy_string']).assign(species_id=lambda x: x.species_id.astype(str)).set_index('species_id').taxonomy_string
species_taxonomy = species_taxonomy.str.split(';').to_frame()

for level_name, level_number in [('p__', 2), ('f__', 5), ('s__', 7)]:
    species_taxonomy = species_taxonomy.assign(**{level_name: species_taxonomy['taxonomy_string'].apply(lambda x: ';'.join(x[:level_number]))})
    
species_taxonomy

In [ ]:
strain_depth_with_minor = pd.read_table(
    'data/ucfmt.a.r.proc.gtpro.filt-poly05-cvrg05.fit-sfacts2-s200-g10000-seed0.collapse-10.strain_depth.tsv',
    # names=['library_id', 'species_strain_id', 'depth'],
    index_col=['sample', 'strain'],
).squeeze().unstack('strain', fill_value=0)

In [ ]:
strain_taxonomy = strain_depth_with_minor.columns.to_series().str.split('-').str[0].to_frame(name='species_id').join(species_taxonomy, on='species_id')

In [ ]:
species_depth = pd.read_table('data/ucfmt.a.r.proc.gtpro.species_depth.tsv', index_col=['sample', 'species_id']).squeeze().unstack('species_id', fill_value=0)
plt.hist(strain_depth_with_minor.sum(1) - species_depth.sum(1))

In [ ]:
thresh = 0.5

strain_collapse = strain_depth_with_minor.columns.to_series()
strain_other = strain_collapse.str.rsplit('-', 1).str[0] + '-other'
strain_collapse = strain_collapse.where(strain_depth_with_minor.max() > thresh, strain_other)

strain_collapse.value_counts().shape

In [ ]:
strain_depth = strain_depth_with_minor.groupby(strain_collapse, axis='columns').sum()

In [ ]:
plt.hist(strain_depth_with_minor.sum(1) - strain_depth.sum(1))

In [ ]:
strain_rabund = strain_depth.divide(strain_depth.sum(1), axis=0)

In [ ]:
plt.hist(np.log10(strain_rabund.max()), bins=np.linspace(-5, 0, num=51))
None

In [ ]:
mgen_meta_baseline = mgen_meta.loc[strain_rabund.index].groupby('subject_id').apply(lambda d: d.sort_values('week_number').head(1)).reset_index('subject_id', drop=True)
baseline_library = mgen_meta_baseline.index.to_series()

In [ ]:
_depth = strain_depth
_n_taxa = len(_depth.columns)
_rabund = _depth.divide(_depth.sum(1), axis=0)

c = 'tab:blue'

_prevalence = (_rabund > 1e-5).mean()
_mean_rabund = _rabund.mean()
_decreasing_prevalence = (_rabund > 1e-5).mean().sort_values(ascending=False).index
_quantile_rabund = _rabund.loc[:, _decreasing_prevalence].cumsum(1).quantile([0.0, 0.05, 0.25, 0.5, 0.75, 0.95, 1.0]).T

xx = np.arange(_n_taxa)
plt.plot(xx, _quantile_rabund[0.5], c=c, lw=2, label='median_rabund')
plt.fill_between(xx, _quantile_rabund[0.0], _quantile_rabund[1.0], color=c, alpha=0.05, edgecolor=None)
plt.fill_between(xx, _quantile_rabund[0.05], _quantile_rabund[0.95], color=c, alpha=0.2, edgecolor=None)
plt.fill_between(xx, _quantile_rabund[0.25], _quantile_rabund[0.75], color=c, alpha=0.2, edgecolor=None)

plt.axvline((_quantile_rabund[0.0] < 0.99).sum(), linestyle='--', lw=1, color='grey')

print('Unique species:', len(set(map(lambda s: s.split('-')[0], idxwhere(_quantile_rabund[0.0] < 0.99)))))